# ASCII File Tree generator
TODO: Helpful Error messages. For example: "variable-name: expected type, got type"

In [ ]:
import os

# walk all directories and subdirectories, generate nested list of files
def walk_dir(path):
    files = {}
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            files[file] = walk_dir(file_path)
        else:
            files[file_path] = None
    return files

# generate ASCII tree from nested list with box-drawing characters
def generate_tree(input_tree, folder_format='{}/', file_format='{}', middle_indent='├── ', passing_indent='│   ', last_indent='└── ', empty_indent='    '):
    """
    Prints an ASCII file tree with box-drawing characters from a nested dictionary.

    Args:
        input_tree (dict): A nested dictionary of files and folders.
        level (int): The current level of the tree.
        folder_format (str): The format of the folder name. Must have exactly one {}.
        file_format (str): The format of the file name. Must have exactly one {}.
        middle_indent (str): The indent for a middle node.
        passing_indent (str): The indent for a passing node.
        last_indent (str): The indent for a last node.
        empty_indent (str): The indent for an empty node.

    Returns:
        str: The ASCII file tree.

    Raises:
        ValueError: If the folder format does not have exactly one {}.
        ValueError: If the file format does not have exactly one {}.
        ValueError: If the indents are not all the same length.
    """
    # handle bad inputs
    if folder_format.count('{}') != 1:
        raise ValueError('The provided folder format does not have exactly one {}')
    if file_format.count('{}') != 1:
        raise ValueError('The provided file format does not have exactly one {}')
    if not (len(middle_indent) == len(passing_indent) == len(last_indent) == len(empty_indent)):
        raise ValueError('The provided indents are not all the same length')

    # separate tree into folders and files
    folders = {os.path.basename(name): content for name, content in input_tree.items() if isinstance(content, dict)}
    files = [os.path.basename(name) for name, content in input_tree.items() if content is None]

    # return shallow tree if all folders are empty
    if all(content == {} for content in folders.values()):
        tree = ''.join(folder_format.format(folder) + '\n' for folder in sorted(folders.keys()))
        tree += ''.join(file_format.format(file) + '\n' for file in sorted(files))
        return tree

    # go through all folders
    tree = ''
    for folder, content in folders.items():
        tree += folder_format.format(folder) + '\n'

        # get the subtree of the folder and add the right indents
        if len(content) > 0:
            subtree = generate_tree(content, folder_format, file_format, middle_indent, passing_indent, last_indent, empty_indent)
            lines = subtree.splitlines()

            # get all line numbers of immediate folders
            folder_indices = [i for i, line in enumerate(lines)
                              if line.startswith(folder_format.split('{}')[0])
                              and not line.startswith(middle_indent)
                              and not line.startswith(passing_indent)
                              and not line.startswith(last_indent)
                              and not line.startswith(empty_indent)
                              and line.endswith(folder_format.split('{}')[1])]
            
            # fill it up, if no folder was found
            if folder_indices == []:
                folder_indices = [-1]
            
            for line_index, line in enumerate(lines):
                # handle a folder
                if line_index in folder_indices:
                    if line_index < folder_indices[-1]:
                        tree += middle_indent + line + '\n'
                    else:
                        tree += last_indent + line + '\n'
                # handle a file
                else:
                    if line_index > folder_indices[-1]:
                        tree += empty_indent + line + '\n'
                    else:
                        tree += passing_indent + line + '\n'
    
    # go through all files
    for file in files:
        tree += file_format.format(file) + '\n'

    # return the generated tree
    return tree

files = walk_dir('.')
print(generate_tree(files))